# SATR AI Co.,Ltd. Practical Test
Current Date and Time (UTC): 2025-02-17 17:20:35
Current User's Login: sesiii

## Overview
- Electricity demand forecasting using data from Kansai Electric Power (JP)
- Target period: January 1, 2023, 00:00:00 – December 31, 2023, 23:00:00
- Using weather data from 8 different cities and historical demand data

In [1]:
# Install required packages
!pip install pandas numpy tensorflow scikit-learn matplotlib jpholiday

In [2]:
# Import libraries and suppress warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import glob
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
import logging
import pickle
import absl.logging

# Suppress all warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')
absl.logging.set_verbosity(absl.logging.ERROR)

# Suppress specific TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

2025-02-17 23:57:45.234962: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 23:57:45.312901: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-17 23:57:45.388431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739816865.451425   41652 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739816865.468967   41652 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 23:57:45.624990: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
from google.colab import files

# Creates directories
!mkdir -p data/electricity_demand data/Weather

print("Please upload demand.csv")
uploaded = files.upload()
!mv demand.csv data/electricity_demand/

# Upload weather data
print("Please upload weather data files for all 8 cities")
uploaded = files.upload()
for file in uploaded.keys():
    if file.endswith('.csv'):
        !mv "$file" data/Weather/

ModuleNotFoundError: No module named 'google.colab'

## Model Development for Electricity Demand Forecasting

### Method Selection and Justification
- **Chosen Method**: LSTM (Long Short-Term Memory) Neural Network
- **Justification**:
  - Capable of capturing long-term dependencies in time series data
  - Well-suited for sequential data with temporal patterns
  - Can handle multiple input features (weather data from 8 cities)
  - Effective at learning seasonal and daily patterns in electricity demand

### Feature Engineering
1. **Temporal Features**:
   - Hour of day (captures daily patterns)
   - Day of week (captures weekly patterns)
   - Month (captures seasonal patterns)
   - Weekend flag (captures weekend vs weekday differences)
   - Holiday flag (using jpholiday library)

2. **Weather Features**:
   - Temperature (primary driver of electricity demand)
   - Humidity (affects perceived temperature)
   - Wind direction (encoded using cardinal directions)
   - Precipitation and snowfall (impact on energy usage)

In [21]:
def load_and_preprocess_weather_data(directory='data/Weather'):
    """Load and preprocess weather data from multiple cities"""
    weather_files = glob.glob(os.path.join(directory, '*.csv'))
    all_weather_data = {}

    wind_mapping = {
        '南': 0, '南南西': 1, '南南東': 2, '西': 3,
        '北西': 4, '南西': 5, '西北西': 6, '北': 7,
        '東': 8, '南東': 9, '北東': 10, '東北東': 11
    }

    for file in weather_files:
        city_name = os.path.basename(file).replace('.csv', '')
        df = pd.read_csv(file)
        df['datetime'] = pd.to_datetime(df['datetime'])
        df.set_index('datetime', inplace=True)

        df['wind_direction'] = df['wind_direction'].map(wind_mapping).fillna(0)

        for col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(df[col].mean())

        # Add city prefix to columns
        df.columns = [f'{city_name}_{col}' for col in df.columns]
        all_weather_data[city_name] = df

    # Combine all weather data
    combined_weather = pd.concat(all_weather_data.values(), axis=1)
    print(f"Loaded weather data from {len(all_weather_data)} cities")

    # Handle any remaining NaN values
    nan_cols = combined_weather.columns[combined_weather.isna().any()].tolist()
    if nan_cols:
        print("Columns with NaN values:", nan_cols)
        combined_weather = combined_weather.fillna(method='ffill').fillna(method='bfill')

    return combined_weather

def prepare_data(demand_df, weather_df, sequence_length=24):
    """Prepare data for model training with additional checks"""
    # Create time features
    data = pd.DataFrame(index=demand_df.index)
    data['demand'] = demand_df['actual_performance']
    data['hour'] = data.index.hour
    data['day_of_week'] = data.index.dayofweek
    data['month'] = data.index.month
    data['is_weekend'] = data.index.dayofweek.isin([5, 6]).astype(int)

    # Add weather features
    for col in weather_df.columns:
        data[col] = weather_df[col]

    # Handle any remaining missing values
    if data.isna().any().any():
        print("Handling remaining missing values if any....")
        data = data.fillna(data.mean())

    # Print data statistics
    print("\nData Statistics:")
    print(data.describe())

    # Handle infinite values
    if np.isinf(data.values).any():
        print("Replacing infinite values with mean values....")
        data = data.replace([np.inf, -np.inf], np.nan)
        data = data.fillna(data.mean())

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(len(scaled_data) - sequence_length):
        X.append(scaled_data[i:(i + sequence_length)])
        y.append(scaled_data[i + sequence_length, 0])

    X = np.array(X)
    y = np.array(y)

    if np.isnan(X).any():
        print("Warning: NaN values found in X after preprocessing")
        X = np.nan_to_num(X, nan=0.0)
    if np.isnan(y).any():
        print("Warning: NaN values found in y after preprocessing")
        y = np.nan_to_num(y, nan=0.0)

    return X, y, scaler, data.columns

def create_model(input_shape):
    """Create an improved model architecture"""
    model = Sequential([
        tf.keras.Input(shape=input_shape),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae', 'mse']
    )
    return model

def train_model(model, X_train, y_train, X_val, y_val):
    """Enhanced training function"""
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=0.0001
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.h5',
            monitor='val_loss',
            save_best_only=True
        )
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        batch_size=32,
        callbacks=callbacks,
        verbose=1
    )
    return history

def analyze_feature_importance(model, X, y_test, feature_names):
    """Analyze feature importance using permutation importance"""
    base_mae = model.evaluate(X, y_test, verbose=0)[1]
    importance = []

    for i in range(X.shape[2]):
        X_temp = X.copy()
        X_temp[:, :, i] = np.random.permutation(X_temp[:, :, i])
        new_mae = model.evaluate(X_temp, y_test, verbose=0)[1]
        importance.append((feature_names[i], new_mae - base_mae))

    importance.sort(key=lambda x: x[1], reverse=True)

    plt.figure(figsize=(12, 6))
    features, scores = zip(*importance[:10])  # Top 10 features
    plt.bar(features, scores)
    plt.xticks(rotation=45, ha='right')
    plt.title('Top 10 Most Important Features')
    plt.tight_layout()
    plt.show()

In [ ]:
import pickle
def main():
    # Print runtime information
    current_datetime = "2025-02-17 18:17:03"
    current_user = "sesiii"

    # Load demand data
    demand_df = pd.read_csv('data/electricity_demand/demand.csv')
    demand_df['datetime'] = pd.to_datetime(demand_df['datetime'])
    demand_df.set_index('datetime', inplace=True)
    print("Demand data shape:", demand_df.shape)

    # Load weather data from all cities
    weather_df = load_and_preprocess_weather_data()
    print("Weather data shape:", weather_df.shape)

    # Prepare data
    X, y, scaler, feature_names = prepare_data(demand_df, weather_df)

    # Print data shapes and check for NaN values
    print("\nFinal Data Shapes:")
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    print("NaN in X:", np.isnan(X).any())
    print("NaN in y:", np.isnan(y).any())

    # Split data
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Further split training data for validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )

    # Create and train model
    model = create_model((X.shape[1], X.shape[2]))
    print("\nModel Summary:")
    model.summary()

    # Train model
    history = train_model(model, X_train, y_train, X_val, y_val)

    # Make predictions
    y_pred = model.predict(X_test)

    # Inverse transform predictions and actual values
    pred_data = np.zeros((len(y_pred), len(feature_names)))
    pred_data[:, 0] = y_pred.flatten()
    y_pred_inv = scaler.inverse_transform(pred_data)[:, 0]

    test_data = np.zeros((len(y_test), len(feature_names)))
    test_data[:, 0] = y_test
    y_test_inv = scaler.inverse_transform(test_data)[:, 0]

    # Calculate metrics
    mse = np.mean((y_pred_inv - y_test_inv) ** 2)
    mae = np.mean(np.abs(y_pred_inv - y_test_inv))
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100

    # Print detailed metrics
    print("\nDetailed Metrics:")
    print(f"Mean Squared Error: {mse:.2f} kW²")
    print(f"Mean Absolute Error: {mae:.2f} kW")
    print(f"Root Mean Squared Error: {rmse:.2f} kW")
    print(f"Mean Absolute Percentage Error: {mape:.2f}%")

    # Plot results with confidence intervals
    plt.figure(figsize=(15, 6))
    pred_std = np.std(y_pred_inv - y_test_inv)
    plt.fill_between(
        range(100),
        y_pred_inv[:100] - 1.96 * pred_std,
        y_pred_inv[:100] + 1.96 * pred_std,
        alpha=0.2,
        color='blue',
        label='95% Confidence Interval'
    )

    # Save results and model
    results = {
        "runtime_info": {
            "datetime": current_datetime,
            "user": current_user
        },
        "model_history": history.history,
        "predictions": y_pred_inv,
        "actual_values": y_test_inv,
        "metrics": {
            "mse": mse,
            "mae": mae,
            "rmse": rmse,
            "mape": mape
        },
        "feature_names": feature_names.tolist()
    }

    # Save results to pickle file
    with open('forecast_results.pkl', 'wb') as f:
        pickle.dump(results, f)

    # Save model in .keras format
    model.save('electricity_demand_model.keras')

    # Download files if in Colab environment
    try:
        from google.colab import files
        files.download('forecast_results.pkl')
        files.download('electricity_demand_model.keras')
        print("\nFiles downloaded successfully!")
    except:
        print("\nFiles saved locally (not running in Colab)")

    plt.plot(y_test_inv[:100], label='Actual', color='blue', alpha=0.7)
    plt.plot(y_pred_inv[:100], label='Predicted', color='red', alpha=0.7)
    plt.legend()
    plt.title('Electricity Demand Forecasting Results with Confidence Intervals')
    plt.xlabel('Time Steps')
    plt.ylabel('Demand (kW)')
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss', color='blue', alpha=0.7)
    plt.plot(history.history['val_loss'], label='Validation Loss', color='red', alpha=0.7)
    plt.title('Model Loss During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Analyze feature importance
    analyze_feature_importance(model, X_test, y_test, feature_names)

if __name__ == "__main__":
    main()

## Evaluation of Forecasting Performance

### Selected Metrics:
1. **Mean Absolute Error (MAE)**
   - Directly interpretable in the same units as demand
   - Less sensitive to outliers than MSE

2. **Root Mean Square Error (RMSE)**
   - Penalizes larger errors more heavily
   - Useful for comparing different models

3. **Mean Absolute Percentage Error (MAPE)**
   - Provides relative error measurement
   - Easy to communicate to non-technical stakeholders

## Error Analysis and Model Challenges

### Identified Issues:
1. **Data Quality**:
   - Missing values in weather data
   - Potential noise in measurements

2. **Model Limitations**:
   - Limited forecast horizon (24 hours)
   - Sensitivity to sudden weather changes
   - Computational resource requirements

## Hypotheses for Accuracy Improvement

1. **Feature Engineering**:
   - Include holiday indicators
   - Add lagged demand values
   - Create interaction features between temperature and time

2. **Model Enhancements**:
   - Implement attention mechanism
   - Use ensemble methods
   - Increase model capacity

## Expected Benefits of Model Deployment

1. **Operational Efficiency**:
   - Better resource allocation
   - Reduced energy waste
   - Improved grid stability

2. **Cost Savings**:
   - Optimized power generation
   - Reduced peak demand charges
   - Better maintenance scheduling

3. **Environmental Impact**:
   - Lower carbon emissions
   - More efficient renewable energy integration